In [23]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated, Literal
from pydantic import Field, BaseModel
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

True

In [24]:
def get_gemini_model():
    model = ChatGoogleGenerativeAI(
        google_api_key=os.getenv("GOOGLE_API_KEY"),
        model="gemini-2.5-flash",
        temperature=0,
    )
    return model


llm = get_gemini_model()

In [25]:
class SentimentSchema(BaseModel):
    sentiment: Literal["positive", "negative"] = Field(
        description="The sentiment of the review"
    )


llm_with_sentiment = llm.with_structured_output(SentimentSchema)


class DiagnosisSchema(BaseModel):
    issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(
        description="The type of issue"
    )
    tone: str = Field(description="The tone of the review")
    urgency: Literal["low", "medium", "high"] = Field(
        description="The urgency of the issue"
    )


llm_with_diagnosis = llm.with_structured_output(DiagnosisSchema)

In [26]:
class ReviewState(TypedDict):
    review: str
    sentiment: Literal["positive", "negative"]
    diagnosis: dict
    response: str

In [27]:
# Functions for nodes
def find_sentiment(state: ReviewState) -> ReviewState:
    prompt = f"Determine the sentiment of the given review: {state['review']}. Respond with 'positive' or 'negative'."
    sentiment = llm_with_sentiment.invoke(prompt).sentiment
    return {"sentiment": sentiment}


def positive_response(state: ReviewState) -> ReviewState:
    prompt = f"Write a warm thank-you message in response to this review: {state['review']}. Also, kindly ask the customer to consider leaving a 5-star review on the website."
    response = llm.invoke(prompt).content
    return {"response": response}


def negative_response(state: ReviewState) -> ReviewState:
    prompt = f"Write a sympathetic apology message in response to this negative review: {state['review']}. Assure the customer that their concerns are being addressed and that a team member will reach out to them shortly."
    response = llm.invoke(prompt).content
    return {"response": response}


def run_diagnosis(state: ReviewState) -> ReviewState:
    prompt = f"Analyze the following negative review and provide a structured diagnosis including issue type, tone, and urgency: {state['review']}."
    diagnosis = llm_with_diagnosis.invoke(prompt).model_dump()
    return {"diagnosis": diagnosis}


def check_sentiment(
    state: ReviewState,
) -> str:
    if state["sentiment"] == "positive":
        return "Positive"
    else:
        return "Negative"

In [28]:
graph = StateGraph(ReviewState)

# nodes
graph.add_node("Find Sentiment", find_sentiment)
graph.add_node("Run Diagnosis", run_diagnosis)
graph.add_node("Positive Response", positive_response)
graph.add_node("Negative Response", negative_response)

# edges
graph.add_edge(START, "Find Sentiment")
graph.add_conditional_edges("Find Sentiment", check_sentiment, {"Positive": "Positive Response", "Negative": "Run Diagnosis"})
graph.add_edge("Positive Response", END)
graph.add_edge("Run Diagnosis", "Negative Response")
graph.add_edge("Negative Response", END)

workflow = graph.compile()

In [29]:
initial_state = {
    "review": "I'm really disappointed with this app. It crashes frequently, is very slow to load, and the user interface is confusing. I also had trouble getting support when I reached out for help."
}
workflow.invoke(initial_state)

{'review': "I'm really disappointed with this app. It crashes frequently, is very slow to load, and the user interface is confusing. I also had trouble getting support when I reached out for help.",
 'sentiment': 'negative',
 'diagnosis': {'issue_type': 'Bug', 'tone': 'Disappointed', 'urgency': 'high'},
 'response': "We are so genuinely sorry to hear about your incredibly frustrating experience with our app. Your detailed feedback is extremely important to us, and we truly regret that we've fallen short of your expectations.\n\nThe frequent crashes, slow loading times, and confusing user interface you've described are absolutely not the experience we want for our users. We are especially concerned to hear that you had trouble getting support when you reached out for help – that is unacceptable.\n\nPlease know that your concerns are being taken very seriously. Our development and product teams are actively investigating these issues as top priorities, and we are committed to resolving t